In [1]:
import os
import pylas
import numpy as np
import pandas as pd
from tqdm import tqdm
import geopandas as gpd
from scipy.stats import kurtosis, skew, variation, iqr
pd.set_option('display.max_columns', None)

# Define the percentiles list
percentiles = [1, 5, 10, 20, 25, 30, 40, 50, 60, 70, 75, 80, 90, 95, 99]

In [2]:
# Read the las file
laz_file_path = '../example_data/LiDAR_example.las'
las = pylas.read(laz_file_path)

# Read the file with bounding boxes
gjson = '../example_data/bboxes_example.geojson'
gdf = gpd.read_file(gjson)

In [3]:
# Import necessary libraries
import numpy as np
import pandas as pd
from scipy.stats import variation, kurtosis, skew, iqr
from tqdm import tqdm

# Declare a list to save DataFrames for each tree
dfs = []

# Compute the actual X, Y, Z coordinates using the LAS file's header offsets and scales
las_X = las.header.x_offset + las.points['X'] * las.header.x_scale
las_Y = las.header.y_offset + las.points['Y'] * las.header.y_scale
las_Z = las.header.z_offset + las.points['Z'] * las.header.z_scale

# Extract the intensity values from the LAS file
Intensity = las.points['intensity']

# Create an index of points classified as vegetation (classification code 1)
veg_index = np.where((las.classification == 1))

# Define the data types for the numpy structured array to hold vegetation points
veg_dtype = [('X', float), ('Y', float), ('Z', float),
             ('Int', np.uint16), ('RN', np.uint8), ('NoRs', np.uint8)]

# Initialize a numpy structured array with zeros to hold the vegetation points
veg = np.zeros(len(veg_index[0]), dtype=veg_dtype)

# Fill the structured array with the filtered vegetation points based on the index
veg['X'] = las_X[veg_index]
veg['Y'] = las_Y[veg_index]
veg['Z'] = las_Z[veg_index]
veg['Int'] = Intensity[veg_index]
veg['RN'] = las.return_number[veg_index]
veg['NoRs'] = las.number_of_returns[veg_index]

# Iterate over each tree in the GeoDataFrame (gdf) and process it
for index, tree in tqdm(gdf.iterrows(), total=len(gdf), desc='Processing Trees'):
    # Get the bounding box of the tree geometry
    min_x, min_y, max_x, max_y = tree.geometry.bounds
    # Calculate the center and radius of the tree (assuming it to be circular)
    center_x, center_y = (min_x + max_x) / 2, (min_y + max_y) / 2
    radius = min((max_x - min_x) / 2, (max_y - min_y) / 2)

    # Filter vegetation points that lie within the bounding box of the tree
    veg_sub = veg[(veg['X'] >= min_x) & (veg['Y'] >= min_y) &
                  (veg['X'] <= max_x) & (veg['Y'] <= max_y)]
    # Further filter points to those within the circular region around the tree center
    veg_sub = veg_sub[(veg_sub['X'] - center_x)**2 + 
                      (veg_sub['Y'] - center_y)**2 <= radius**2]

    # Normalize the Z values by subtracting the minimum Z value (ground level)
    z_min = min(veg_sub["Z"])
    Z_normalized = veg_sub["Z"] - z_min

    # Apply a filter to remove points below a certain height (2 meters)
    filter_value = 2
    X_filter = veg_sub["X"][Z_normalized >= filter_value]
    Y_filter = veg_sub["Y"][Z_normalized >= filter_value]
    Z_filter = Z_normalized[Z_normalized >= filter_value]
    Int_filter = veg_sub["Int"][Z_normalized >= filter_value]
    RN_filter = veg_sub["RN"][Z_normalized >= filter_value]
    NoRs_filter = veg_sub["NoRs"][Z_normalized >= filter_value]

    # Get the coordinates of the tree's centroid (rounded to 3 decimal places)
    x_coord = round(tree.geometry.centroid.x, 3)
    y_coord = round(tree.geometry.centroid.y, 3)

    # Initialize a dictionary to store various metrics for the tree
    metrics = {}

    '''---------------------- Geometric metrics (Height) ---------------------'''
    # Maximum height
    metrics['H_max'] = np.max(Z_filter)
    # Median height
    metrics['H_med'] = np.median(Z_filter)
    # Minimum height
    metrics['H_min'] = np.min(Z_filter)
    # Mean height
    metrics['H_mean'] = np.mean(Z_filter)
    # Standard deviation of height
    metrics['H_std'] = np.std(Z_filter)
    # Variance of height
    metrics['H_var'] = np.var(Z_filter)
    # Coefficient of variation of height
    metrics['H_cv'] = variation(Z_filter)
    # Kurtosis of variation of height
    metrics['H_kurt'] = kurtosis(Z_filter)
    # Skewness of variation of height
    metrics['H_skew'] = skew(Z_filter)
    # Mean height of first-or-single returns (return number 1)
    metrics['H_mean_first'] = np.mean(Z_filter[RN_filter == 1])
    # Mean height of single returns (points with only one return)
    metrics['H_mean_single'] = np.mean(Z_filter[NoRs_filter == 1]) if np.any(NoRs_filter == 1) else 0
    # Percentage of first returns
    metrics['H_total_returns_first'] = np.sum(Z_filter[RN_filter == 1]) / np.sum(Z_filter) * 100
    # Percentage of last returns (points where return number equals the number of returns)
    metrics['H_total_returns_last'] = np.sum(Z_filter[RN_filter == NoRs_filter]) / np.sum(Z_filter) * 100
    # All returns (sum of Z values divided by the number of points)
    metrics['H_all_returns'] = np.sum(Z_filter)/len(Z_filter)
    # Canopy relief ratio = (Elev.mean − Elev.min)/(Elev.max – Elev.min)
    metrics['H_CNR'] = (metrics['H_mean']-metrics['H_min'])/(metrics['H_max']-metrics['H_min'])
    # Average Absolute Deviation of Height
    metrics['H_AAD'] = np.mean(np.abs(Z_filter - metrics['H_mean']))

    # Compute height percentiles (e.g., 25th, 50th, 75th)
    Z_filter_sorted = np.sort(Z_filter)
    for p in percentiles:
        metrics[f'Hp_{p}%'] = np.percentile(Z_filter_sorted, p)
    # Interquartile range (IQR) of height
    metrics['Hp_IQR'] = iqr(Z_filter_sorted)

    # Compute Accumulated Incremental Height (AIH)
    cumulative_Z = np.cumsum(Z_filter_sorted)
    for p in percentiles:
        metrics[f'AIH_{p}%'] = np.percentile(cumulative_Z, p)
    # Interquartile range (AIH_IQR) of accumulated height
    metrics['AIH_IQR'] = iqr(cumulative_Z)

    # Compute height density metrics by dividing the height range into 10 slices
    height_slices = np.linspace(metrics['H_min'], metrics['H_max'], 11)
    H_density_vars = np.histogram(Z_filter, bins=height_slices)[0] / len(Z_filter)
    for count, H_density_var in enumerate(H_density_vars):
        metrics[f'H_density_var_{count+1}%'] = H_density_var

    # Median Absolute Deviation of height from the median
    metrics['H_MADMedian'] = np.median(np.abs(Z_filter - metrics['H_med']))
    # Generalized means for the 2nd power (quadratic mean or RMS)
    metrics['H_squared_mean'] = np.sqrt(np.mean(Z_filter ** 2))
    # Generalized means for the 3rd power (cubic mean)
    metrics['H_cubed_mean'] = np.power(np.mean(Z_filter ** 3), 1/3)

    '''---------------------- Radiometric metrics (Intensity) ---------------------'''
    # Maximum intensity
    metrics['I_max'] = np.max(Int_filter)
    # Median intensity
    metrics['I_med'] = np.median(Int_filter)
    # Minimum intensity
    metrics['I_min'] = np.min(Int_filter)
    # Mean intensity
    metrics['I_mean'] = np.mean(Int_filter)
    # Standard deviation of intensity
    metrics['I_std'] = np.std(Int_filter)
    # Variance of intensity
    metrics['I_var'] = np.var(Int_filter)
    # Coefficient of variation of intensity
    metrics['I_cv'] = variation(Int_filter)
    # Kurtosis of variation of intensity
    metrics['I_kurt'] = kurtosis(Int_filter)
    # Skewness of variation of intensity
    metrics['I_skew'] = skew(Int_filter)
    # Mean intensity of first-or-single returns
    metrics['I_mean_first'] = np.mean(Int_filter[RN_filter == 1])
    # Mean intensity of single returns
    metrics['I_mean_single'] = np.mean(Int_filter[NoRs_filter == 1]) if np.any(NoRs_filter == 1) else 0
    # Percentage of first returns (intensity)
    metrics['I_total_returns_first'] = np.sum(Int_filter[RN_filter == 1]) / np.sum(Int_filter) * 100
    # Percentage of last returns (intensity)
    metrics['I_total_returns_last'] = np.sum(Int_filter[RN_filter == NoRs_filter]) / np.sum(Int_filter) * 100
    # All returns (intensity)
    metrics['I_all_returns'] = np.sum(Int_filter)/len(Int_filter)
    # Canopy relief ratio for intensity
    metrics['I_CNR'] = (metrics['I_mean']-metrics['I_min'])/(metrics['I_max']-metrics['I_min'])
    # Average Absolute Deviation of intensity
    metrics['I_AAD'] = np.mean(np.abs(Int_filter - metrics['I_mean']))       

    # Compute intensity percentiles
    Int_filter_sorted = np.sort(Int_filter)
    for p in percentiles:
        metrics[f'Ip_{p}%'] = np.percentile(Int_filter_sorted, p)
    # Interquartile range (IQR) of intensity
    metrics['Ip_IQR'] = iqr(Int_filter_sorted)

    # Compute Accumulated Incremental Intensity (AII)
    cumulative_Int = np.cumsum(Int_filter_sorted)
    for p in percentiles:
        metrics[f'AII_{p}%'] = np.percentile(cumulative_Int, p)
    # Interquartile range (AII_IQR) of accumulated intensity
    metrics['AII_IQR'] = iqr(cumulative_Int)

    # Compute intensity density metrics by dividing the intensity range into 10 slices
    intensity_slices = np.linspace(metrics['I_min'], metrics['I_max'], 11)
    I_density_vars = np.histogram(Int_filter, bins=intensity_slices)[0] / len(Int_filter)
    for count, I_density_var in enumerate(I_density_vars):
        metrics[f'I_density_var_{count+1}%'] = I_density_var

    # Median Absolute Deviation of intensity from the median
    metrics['I_MADMedian'] = np.median(np.abs(Int_filter - metrics['I_med']))
    # Generalized means for the 2nd power (quadratic mean or RMS)
    metrics['I_squared_mean'] = np.sqrt(np.mean(Int_filter ** 2))
    # Generalized means for the 3rd power (cubic mean)
    metrics['I_cubed_mean'] = np.power(np.mean(Int_filter ** 3), 1/3)

    # Create a DataFrame from the metrics dictionary for the current tree
    df = pd.DataFrame(metrics, index=[0])
    # Append the DataFrame to the list
    dfs.append(df)

# Combine all metrics DataFrames into one final DataFrame
df_final = pd.concat(dfs, ignore_index=True)

# Display the final DataFrame
df_final

Processing Trees: 100%|█████████████████████████████████████████████████████████████| 125/125 [00:01<00:00, 107.39it/s]


,H_max,H_med,H_min,H_mean,H_std,H_var,H_cv,H_kurt,H_skew,H_mean_first,H_mean_single,H_total_returns_first,H_total_returns_last,H_all_returns,H_CNR,H_AAD,Hp_1%,Hp_5%,Hp_10%,Hp_20%,Hp_25%,Hp_30%,Hp_40%,Hp_50%,Hp_60%,Hp_70%,Hp_75%,Hp_80%,Hp_90%,Hp_95%,Hp_99%,Hp_IQR,AIH_1%,AIH_5%,AIH_10%,AIH_20%,AIH_25%,AIH_30%,AIH_40%,AIH_50%,AIH_60%,AIH_70%,AIH_75%,AIH_80%,AIH_90%,AIH_95%,AIH_99%,AIH_IQR,H_density_var_1%,H_density_var_2%,H_density_var_3%,H_density_var_4%,H_density_var_5%,H_density_var_6%,H_density_var_7%,H_density_var_8%,H_density_var_9%,H_density_var_10%,H_MADMedian,H_squared_mean,H_cubed_mean,I_max,I_med,I_min,I_mean,I_std,I_var,I_cv,I_kurt,I_skew,I_mean_first,I_mean_single,I_total_returns_first,I_total_returns_last,I_all_returns,I_CNR,I_AAD,Ip_1%,Ip_5%,Ip_10%,Ip_20%,Ip_25%,Ip_30%,Ip_40%,Ip_50%,Ip_60%,Ip_70%,Ip_75%,Ip_80%,Ip_90%,Ip_95%,Ip_99%,Ip_IQR,AII_1%,AII_5%,AII_10%,AII_20%,AII_25%,AII_30%,AII_40%,AII_50%,AII_60%,AII_70%,AII_75%,AII_80%,AII_90%,AII_95%,AII_99%,AII_IQR,I_density_var_1%,I_density_var_2%,I_density_var_3%,I_density_var_4%,I_density_var_5%,I_density_var_6%,I_density_var_7%,I_density_var_8%,I_density_var_9%,I_density_var_10%,I_MADMedian,I_squared_mean,I_cubed_mean
0,18.285,11.022,2.045,11.044563,3.230748,10.437732,0.292519,-0.855121,0.007559,12.325003,7.235500,62.116364,0.215975,11.044563,0.554160,2.712761,4.97820,6.13600,6.6088,7.7438,8.47450,9.1212,10.1382,11.022,12.0688,13.0856,13.62950,14.1696,15.2690,16.30200,17.61922,5.15500,124.14242,839.12540,1828.3852,4034.6210,5286.9875,6661.8972,9634.3608,12915.8070,16477.7532,20367.5956,22437.14450,24594.3698,29147.4068,31582.96410,33672.71660,17150.15700,0.003872,0.012262,0.117457,0.119716,0.149403,0.173927,0.154243,0.148112,0.085511,0.035495,2.5860,11.507393,11.919220,1440,468.0,83,480.047112,208.095705,43303.822240,0.433490,-0.368603,0.357285,506.343188,1007.500000,58.712238,0.610218,480.047112,0.292592,173.582812,146.96,178.00,209.0,267.6,302.00,340.0,412.0,468.0,534.8,598.0,631.00,671.0,758.0,825.00,962.02,329.00,3982.06,24573.20,54765.2,128220.8,172381.00,222048.0,338934.4,475419.0,631029.0,806864.8,901810.50,1002614.4,1222755.6,1345293.50,1454651.26,729429.5,0.115198,0.204582,0.212004,0.211359,0.158761,0.075508,0.018393,0.002581,0.000968,0.000645,164.0,182.778802,31.880288
1,16.158,10.132,2.332,10.162971,2.802431,7.853620,0.275749,-0.942378,-0.035953,11.476389,12.808500,63.222308,0.286153,10.162971,0.566395,2.375784,4.56610,5.80050,6.3730,7.3420,7.90450,8.4480,9.3190,10.132,11.1110,11.9990,12.39550,12.9040,13.9630,14.54900,15.46860,4.49100,114.91490,701.59900,1536.7980,3402.0820,4438.3525,5558.7330,7982.4930,10638.6740,13536.6430,16690.7940,18355.94650,20082.8110,23749.9300,25692.11600,27324.18480,13917.59400,0.001831,0.018674,0.092640,0.135115,0.143171,0.158184,0.151959,0.144636,0.116807,0.036983,2.2550,10.542277,10.881228,1190,469.0,80,483.132552,202.675920,41077.528384,0.419504,-0.814800,0.219588,535.827338,817.000000,62.093244,0.523103,483.132552,0.363182,170.873246,141.00,183.00,218.0,274.0,308.50,347.0,417.0,469.0,539.0,603.0,639.00,679.0,758.0,815.00,920.40,330.50,3629.30,21838.50,49307.0,116186.0,155717.50,200573.0,306028.0,427097.0,564355.0,720565.0,805183.50,895492.0,1091357.0,1198716.50,1292490.70,649466.0,0.057488,0.186012,0.145002,0.190773,0.164042,0.140242,0.088246,0.021604,0.005859,0.000732,166.0,181.692238,31.744961
2,16.355,10.549,2.207,10.607694,2.601672,6.768699,0.245263,-0.695063,-0.111251,11.763651,0.000000,66.561863,0.187017,10.607694,0.593773,2.176793,5.38590,6.25850,7.0960,8.1540,8.71650,9.1600,9.8280,10.549,11.5190,12.2860,12.67450,12.9870,14.0230,14.75600,15.65560,3.95800,86.57920,530.68700,1164.5370,2614.1370,3413.1135,4258.1450,6051.1470,7977.1670,10056.7090,12311.0190,13491.82650,14704.1310,17255.3170,18614.99600,19758.13700,10078.71300,0.002644,0.004759,0.053411,0.108937,0.150185,0.195664,0.158117,0.179799,0.108937,0.037546,2.0040,10.922082,11.205645,1099,441.0,75,446.908514,183.761890,33768.432138,0.411185,-0.633217,0.286062,475